In [24]:
# Configurar el path para importar los módulos
import sys
import os

# Añadir el directorio raíz del proyecto al path
root_dir = os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
if root_dir not in sys.path:
    sys.path.append(root_dir)

In [25]:
# Importar las funciones necesarias
from db.conexion_db import crear_conexion, cerrar_conexion
from db.querys_db import insert_data, consult_data
import pandas as pd
import numpy as np

In [26]:
tbl_inventario_mb52 = 'plan_inventario_mb52'
tbl_inventario_ewm = 'plan_inventario_scwm'
tbl_cruce_inventario = 'cruce_inventario'

cabezeras_cruce_inventario = ['union','material','materialtxt','centro','tipomaterial','grupoarticulo','ctd_total_mb52','Producto','propietario','Sin_Asignacion','Ctd_Garantia','Ctd_nuevo','Ctd_reparacion','ctd_total_scwm','diferencia_stocks']

In [27]:
connection = crear_conexion()
consult_data(connection,f"truncate table {tbl_cruce_inventario}")

inventario_mb52 = consult_data(connection,f"select * from {tbl_inventario_mb52}")
inventario_ewm = consult_data(connection,f"select Producto,Propietario,Lote,Tipo_stock,ctd_producto from {tbl_inventario_ewm}")

Conexion Exitosa


In [28]:
inventario_mb = pd.DataFrame(inventario_mb52)
inventario_sc = pd.DataFrame(inventario_ewm)

In [79]:
filtrar_inventario_mb = inventario_mb.loc[inventario_mb['alm'].isin(['0002','0012'])].reset_index(drop='index')
agrupar_inventario_mb = filtrar_inventario_mb.groupby(['material','materialtxt','centro','tipomaterial','grupoarticulo'])['cantidad'].sum().reset_index()
agrupar_inventario_mb['identificador'] = agrupar_inventario_mb['material'] + agrupar_inventario_mb['centro']


In [80]:
filtrar_inventario_sc = inventario_sc.loc[inventario_sc['Tipo_stock'].isin(['S2','M2'])].reset_index()
filtrar_inventario_sc['ctd_producto'] = filtrar_inventario_sc['ctd_producto'].astype(float)
filtrar_inventario_sc.loc[filtrar_inventario_sc['Lote'].isnull() ,'Lote'] = 'Sin lote'
pivotear_inventario_sc = filtrar_inventario_sc.pivot_table(index=['Producto','Propietario'],columns='Lote',values='ctd_producto', aggfunc='sum', fill_value=0)
inv_sc = pivotear_inventario_sc.reset_index()
inv_sc['identificador'] = inv_sc['Producto'] + inv_sc['Propietario']

In [97]:
union_inventarios = agrupar_inventario_mb.merge(inv_sc, how="left", on="identificador")
reorganizar_inv = union_inventarios.reindex(['identificador','material','materialtxt','centro','tipomaterial','grupoarticulo','cantidad','Producto','Propietario','Sin lote','GARAN','NUEVO','REPAR'], axis=1)


In [101]:
reorganizar_inv['total_2'] = reorganizar_inv['Sin lote'] + reorganizar_inv['GARAN'] + reorganizar_inv['NUEVO'] + reorganizar_inv['REPAR']

reorganizar_inv['dif_stock'] = reorganizar_inv['cantidad'].astype(float) - reorganizar_inv['total_2']

In [102]:
display(reorganizar_inv)

,identificador,material,materialtxt,centro,tipomaterial,grupoarticulo,cantidad,Producto,Propietario,Sin lote,GARAN,NUEVO,REPAR,total_2,dif_stock
0,101CG-4-4C154,101CG-4-4,1/4X1/4 101CG-4-4 X 1MTS,C154,ZMER,G112,10.00,101CG-4-4,C154,0.0,0.0,10.0,0.0,10.0,0.0
1,10643-12-12C154,10643-12-12,3/4X3/4 10671-12-12,C154,ZMER,G112,4.00,10643-12-12,C154,0.0,0.0,4.0,0.0,4.0,0.0
2,10643-6-4CFC154,10643-6-4CF,3/8X1/4 10643-6-4,C154,ZMER,G112,12.00,10643-6-4CF,C154,0.0,0.0,12.0,0.0,12.0,0.0
3,10643-6-6C154,10643-6-6,3/8X3/8 10643-6-6,C154,ZMER,G112,43.00,10643-6-6,C154,0.0,0.0,43.0,0.0,43.0,0.0
4,10643-8-8C154,10643-8-8,1/2X1/2 10643-8-8,C154,ZMER,G112,20.00,10643-8-8,C154,0.0,0.0,20.0,0.0,20.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15149,YSM8092014C154,YSM8092014,Pasador,C154,ZMER,G003,2.00,YSM8092014,C154,0.0,0.0,2.0,0.0,2.0,0.0
15150,YSM8096448C154,YSM8096448,Pasador,C154,ZMER,G003,1.00,YSM8096448,C154,0.0,0.0,1.0,0.0,1.0,0.0
15151,YSM8102957C154,YSM8102957,Pasador,C154,ZMER,G003,2.00,YSM8102957,C154,0.0,0.0,2.0,0.0,2.0,0.0
15152,YSMAT21963C154,YSMAT21963,Pasador,C154,ZMER,G003,2.00,YSMAT21963,C154,0.0,0.0,2.0,0.0,2.0,0.0
